### 1) Load necessary Python packages

In [103]:
import os
import sys
import dill

# Add the local src directory to the path
sys.path.append('./src/')

# Load the GBA_model class
from GBA_model import *

### 2) Define a function to load a model from a binary backup
See the script `csv_to_binary.py`.

In [104]:
### Load the model from a binary file ###
def load_model( model_name ):
    filename = "./binary_models/"+model_name+".gba"
    assert os.path.isfile(filename), "ERROR: model not found."
    ifile = open(filename, "rb")
    model = dill.load(ifile)
    ifile.close()
    return model

### 3) Load the model "EC12b" (a very abstract model of <em>E. coli</em>)

In [105]:
# I previously saved the model as a binary file with the script `csv_to_binary.py`
model = load_model("EC12b")

### 4) Define the condition
The condition defines the environmental state of the cell (the composition of the medium). External concentrations are stored in the vector $x$.
Conditions are listed for the EC12b model in `./csv_models/EC12b/conditions.csv`.
We will use the condition 1.

In [106]:
model.set_condition("1")

### 5) Calculate the initial state
This little linear program routine allows us to find at least one suboptimal solution for the GBA model.
The solution (like every solutions) must satisfy the following constraints:
- $c > 0$ (positivity of mass concentrations)
- $p > 0$ (positivity of protein mass concentrations)
- $sM . f = 1$ (Relative density of 1)

The routine requires GUROBIpy to be installed.

In [107]:
model.solve_local_linear_problem()

### 6) Calculate all model variables

In [108]:
model.calculate()

### 7) Display a few variables

In [109]:
print("\n> Summary of the model:")
print(model)


> Summary of the model:

 -------- Model report: EC12b --------
| • Nb metabolites          = 16
| • Nb external metabolites = 4
| • Nb internal metabolites = 12
 -------------------------------------
| • Nb reactions          = 12
| • Nb exchange reactions = 3
| • Nb internal reactions = 8
 -------------------------------------



In [110]:
print("\n> Flux fractions f:")
df = pd.DataFrame(data=model.f, index=model.reaction_ids, columns=["f"])
df


> Flux fractions f:


,f
tC,0.558586
tP,0.220707
tN,0.220707
tAce,0.000000
Resp,0.261373
Ferm,0.041667
Nucprod,0.683019
Aaprod,0.227330
Replication,0.008333
Transcription,0.598050


In [111]:
print("\n> Fluxes v:")
df = pd.DataFrame(data=model.v, index=model.reaction_ids, columns=["v"])
df


> Fluxes v:


,v
tC,25.247762
tP,9.975826
tN,9.975826
tAce,0.000000
Resp,11.813886
Ferm,1.883309
Nucprod,30.872042
Aaprod,10.275180
Replication,0.376662
Transcription,27.031514


In [112]:
print("\n> Mass fractions b:")
df = pd.DataFrame(data=model.b, index=model.c_ids, columns=["b"])
df


> Mass fractions b:


,b
C,0.008333
P,0.008333
N,0.008333
Ace,0.008333
ADP,0.139020
ATP,0.008333
AA,0.008333
Nuc,0.008333
DNA,0.008333
tRNA,0.008333


In [113]:
print("\n> Mass concentrations c:")
df = pd.DataFrame(data=model.c, index=model.c_ids, columns=["c"])
df


> Mass concentrations c:


,c
C,2.833333
P,2.833333
N,2.833333
Ace,2.833333
ADP,47.266667
ATP,2.833333
AA,2.833333
Nuc,2.833333
DNA,2.833333
tRNA,2.833333


In [114]:
print("\n> Growth rate mu:")
print(model.mu)


> Growth rate mu:
0.13293945281963915


### 8) Test the consistency of the model (positivity + total density)
The result of the consistency check is stored in the variable model.consistent

In [115]:
model.check_model_consistency()
print(model.consistent)

True


### 9) Mu derivatives
The function `model.calculate()` also calculates derivatives $\dfrac{d\mu}{df}$.
To follow the fitness gradient, the important variable is `GCC_f`, which corrects for total density.

In [116]:
print("\n> dmu/df:")
df = pd.DataFrame(data=model.dmu_f, index=model.reaction_ids, columns=["dmu/df"])
df


> dmu/df:


,dmu/df
tC,3.601237
tP,2.782637
tN,2.477095
tAce,-0.000615
Resp,5.261154
Ferm,3.569710
Nucprod,-3.325263
Aaprod,-0.736266
Replication,0.122090
Transcription,2.302624


In [117]:
print("\n> GCC_f:")
df = pd.DataFrame(data=model.GCC_f, index=model.reaction_ids, columns=["GCC_f"])
df


> GCC_f:


,GCC_f
tC,0.000000
tP,-0.818601
tN,-1.124143
tAce,-0.000615
Resp,5.261154
Ferm,3.569710
Nucprod,-3.325263
Aaprod,-0.736266
Replication,0.122090
Transcription,2.302624


In [118]:
f_truncIndex = ['tP', 'tN', 'tAce', 'Resp', 'Ferm', 'Nucprod', 'Aaprod', 'Replication', 'Transcription', 'tRNAcharging', 'Ribosome'] #first element index removed
print("\n> f_trunc:")
df = pd.DataFrame(data=model.f_trunc, index=f_truncIndex, columns=["f_trunc"])
df


> f_trunc:


,f_trunc
tP,0.220707
tN,0.220707
tAce,0.000000
Resp,0.261373
Ferm,0.041667
Nucprod,0.683019
Aaprod,0.227330
Replication,0.008333
Transcription,0.598050
tRNAcharging,0.981318


In [119]:
def trajectory(model):
  threshold = 1e-100                   # for gradient '=' 0
  current_f = np.copy(model.f_trunc) # safe current_f, that's consistent
  next_f = np.copy(model.f_trunc)     # the f_trunc, that we are going to change
  print("first next_f: ")
  print(next_f)
  t = 0                              # time
  dt = 20
  max_time = 200
  while (t <= max_time or ( np.abs(model.GCC_f).all() < threshold == False ) ): # end loop if time is up or if every entry of GCC_f is "0"
    next_f = next_f + model.GCC_f[1:] *dt                                       # add without first index
    model.calculate()                                                           #calculate everything
    print("-----------------")
    print ("GCC_f ")
    print(model.GCC_f[1:])
    print("-----------------")
    print("next_f: ")
    print(next_f)
    
    model.check_model_consistency()     #check consistency
    if (model.consistent):
      t = t + dt                     # calc. new t
      model.set_f(current_f)        # set consistent f to the models actual f
      current_f = next_f            # not sure, if I should change current_f
    else:
      next_f = current_f
      if (dt > 1e-100):             # make sure dt is not too small
       dt = dt - 0.005
       t = t + dt                   # calc. new t

  return model.f, model.GCC_f, model.consistent

trajectory(model)

first next_f: 
[0.22070697 0.22070697 0.         0.26137255 0.04166667 0.68301864
 0.22732994 0.00833333 0.59805011 0.98131808 0.97205882]
-----------------
GCC_f 
[-8.18600637e-01 -1.12414270e+00 -6.14941992e-04  5.26115416e+00
  3.56970971e+00 -3.32526265e+00 -7.36266397e-01  1.22090450e-01
  2.30262388e+00  9.28816255e-01 -4.44606430e+00]
-----------------
next_f: 
[-1.61513058e+01 -2.22621471e+01 -1.22988398e-02  1.05484456e+02
  7.14358609e+01 -6.58222344e+01 -1.44979980e+01  2.45014234e+00
  4.66505277e+01  1.95576432e+01 -8.79492272e+01]
-----------------
GCC_f 
[-8.18600637e-01 -1.12414270e+00 -6.14941992e-04  5.26115416e+00
  3.56970971e+00 -3.32526265e+00 -7.36266397e-01  1.22090450e-01
  2.30262388e+00  9.28816255e-01 -4.44606430e+00]
-----------------
next_f: 
[-3.25233185e+01 -4.47450011e+01 -2.45976797e-02  2.10707539e+02
  1.42830055e+02 -1.32327487e+02 -2.92233259e+01  4.89195134e+00
  9.27030052e+01  3.81339683e+01 -1.76870513e+02]
-----------------
GCC_f 
[ 0.00649448

(array([ 3.94134528e+01, -1.61513058e+01, -2.22621471e+01, -1.22988398e-02,
         1.05484456e+02,  7.14358609e+01, -6.58222344e+01, -1.44979980e+01,
         2.45014234e+00,  4.66505277e+01,  1.95576432e+01, -8.79492272e+01]),
 array([ 0.        ,  0.00649448, -0.0087445 ,  0.01848223,  0.09270875,
         0.06200397,  0.09294788,  0.06203237,  0.03915266,  0.03114307,
         0.04621876,  0.11140605]),
 False)